<h2>imports to environment</h2>

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
#from zipfile import ZipFile
import warnings
from pathlib import Path

from lifelines.plotting import plot_lifetimes      # Lifeline package for the Survival Analysis
%pylab inline
from lifelines import KaplanMeierFitter
kmf = KaplanMeierFitter()

Populating the interactive namespace from numpy and matplotlib


<h2> import dataframes</h2>

In [1]:
#creating functions that search parts of strings and clip values
def left(aString, howMany):
    if howMany <1:
        return ''
    else:
        return aString[:howMany]

def right(aString, howMany):
    if howMany <1:
        return ''
    else:
        return aString[-howMany:]

def mid(aString, startChar, howMany):
    if howMany < 1:
        return ''
    else:
        return aString[startChar:startChar+howMany]

def new_columns(df, columnsList):
    df = df.join(pd.DataFrame(str(np.nan), index=df.index, dtype='str',columns=columnsList))
    return df

def print_header(text_in):
    text_top_bottom = "\n========================================================================"
    len_mid = np.floor((len(text_top_bottom) - len(text_in))/2)
    text_mid_a = ""
    for n in range(int(len_mid)):
        text_mid_a = text_mid_a + "+"
    text_mid = "\n"+text_mid_a + text_in + text_mid_a
    while len(text_top_bottom) < len(text_mid):
        text_top_bottom = text_top_bottom + "="
    text_out = text_top_bottom + text_mid + text_top_bottom + "\n"
    return text_out


def imports():
    global file_path
    global randSample
    #randSample = sorted(sample(range(5000),k=25))
    search_select = input("What is the source of the MOVEMENT data? (1=internet, 2=home Mac, 3=office Windows)")
    import_type = input("What type of import is it? (1 = initial (only cases and movement), 2 = initial (full), 3 = data by week, 4 = merged datasets, 5 = merged and cleaned)")    
    file_path = ""
    print(file_path)
    if search_select == "2":
        file_path = Path('../../../02 data/')
        print(file_path)
    elif search_select == "3":
        file_path = Path('./IN/')
        print(file_path)
    
    if import_type == "1":
        import_movement_bra()
        import_brasilio()
    
    elif import_type == "2":
        import_movement()
        import_brasilio()
        import_responses_souza()
        import_responses_barberia()
        import_recoded()
        import_ibge_codes()
    
    elif import_type == "3":
        import_recoded()
        import_ibge_codes()
    
    elif import_type == "4":
        import_merged()
        import_ibge_codes()
    elif import_type == "5":
        import_cleaned()
        #import_ibge_codes()        

def import_movement():
    global movement
    global search_select
    #loading Movement Range Maps world data for a given date 
    file_name = "WD FACEBOOK Data for Good/movement-range-2021-04-11.csv"
    file_url = file_path / file_name
    movement = pd.read_csv(file_url,delimiter='\t', encoding='utf-8',parse_dates=['ds'])
    print(print_header("Movement Range"), movement.info())
    
def import_brasilio():
    #loading Brasil.io daily cases
    global cases
    global deaths
    file_name = "BR BRASIL.IO/2021-05-25 caso_full.csv"
    file_url = file_path / file_name
    print(file_url)
    cases = pd.read_csv(file_url,parse_dates=['date'])
    print(print_header("Daily Cases (Brasil.io)"), cases.info())

def import_movement_bra():
    global movement_bra
    file_name = "WD FACEBOOK Data for Good/movement-range-2021-04-11_BRA.csv"
    file_url = file_path / file_name
    movement_bra = pd.read_csv(file_url)
    print(print_header("Movement Range"),movement_bra.info())
    
def import_responses_souza():
    global responses_souza
    file_name = "BR SOUZA et al Non-Pharmaceutical Interventions in BR/Dataset_Port_and_Eng.xlsx"
    file_url = file_path / file_name
    responses_souza = pd.read_excel(file_url, sheet_name="Covid-19 NPIs (english)", parse_dates=['Q1. Start date','Q2. Start date', 'Q3. Start date', 'Q4. Start date', 'Q5. Start date', 'Q6. Start date'])
    print(print_header("Responses (Souza et al.)"), responses_souza.info())

def import_responses_barberia():
    global responses_barberia
    file_name = "BR BARBERIA et al RPDS COVID policy responses index/CGRT Municipalities.csv"
    file_url = file_path / file_name
    responses_barberia = pd.read_csv(file_url, parse_dates=['date'], encoding='utf-8')
    print(print_header("Responses, Oxford (Barberia et al.)"), responses_barberia.info())
    
def import_ibge_codes():
    file_path = Path('./OUT/')
    global ibge_codes
    file_name = "IBGE GADM city code compat.csv"
    url = file_path / file_name
    ibge_codes = pd.read_csv(url, delimiter=";", encoding='utf-8', dtype = {'cod_mun_ibge':str,'UF':str},thousands=".", engine='c')
    ibge_codes.columns = ['uf', 'uf_name', 'ibge_code', 'mun_name', 'GADM_GID_2','pop_2020']
    print(print_header("IBGE city codes"),ibge_codes.info())
    
def import_recoded():
    file_path = Path('./OUT/')
    global movement_bra_recoded
    global movement_bra_by_epi_week
    global cases_recoded
    global cases_by_epi_week
    
    #movement range
    file_name = 'CVGI_FB_movement_bra_recoded.csv'
    url = file_path / file_name
    movement_bra_recoded = pd.read_csv(url, encoding="utf_8",parse_dates=['ds'],
                                       dtype = {'polygon_id':str, 'epi_week':str, 'all_day_bing_tiles_visited_relative_change':np.float64,
                                                'all_day_ratio_single_tile_users':np.float64}, engine='c')
    movement_bra_recoded.drop("Unnamed: 0", axis=1, inplace=True)
    movement_bra_recoded.columns = ['ds', 'GADM_GID_2', 'name_mun','movement_change',
                                'stay_put', 'epi_week']
    print(print_header("Movement Range, recoded"), movement_bra_recoded.info())
    
    file_name = 'CVGI_FB_movement_bra_by_epi_week.csv'
    url = file_path / file_name
    movement_bra_by_epi_week = pd.read_csv(url, encoding="utf_8", dtype = {'polygon_id':str, 'epi_week':str, 
                                                                           'all_day_bing_tiles_visited_relative_change':np.float64,
                                                                           'all_day_ratio_single_tile_users':np.float64, 
                                                                           'epi_week_col':str}, engine='c')
    print(print_header("Movement Range, by epidemiological week"), movement_bra_by_epi_week.info())
    
    #cases
    file_name = 'CVGI_FB_cases_recoded.csv'
    url = file_path / file_name
    cases_recoded = pd.read_csv(url, encoding="utf_8",parse_dates=['date'],dtype = {'city_ibge_code':str, 'epi_week':str}, engine='c')
    print(print_header("Cases, recoded"), cases_recoded.info())
    
    file_name = 'CVGI_FB_cases_by_epi_week.csv'
    url = file_path / file_name
    cases_by_epi_week = pd.read_csv(url, encoding="utf_8", dtype = {'city_ibge_code':str, 'epi_week':str, 'acc_cases':int, 
                                                                    'acc_deaths':int, 'new_cases':int,'new_deaths':int, 
                                                                    'pop_2019': np.float64, 'epi_week_col':str, 'acc_cases_100k': np.float64,
                                                                    'acc_deaths_1mi': np.float64}, engine='c')
    cases_by_epi_week.dropna(axis=0, inplace=True, how="all",subset=['city_ibge_code'])
    cases_by_epi_week.columns = ['ibge_code', 'epi_week', 'acc_cases', 'acc_deaths', 'new_cases','new_deaths', 'pop_2019',
                             'epi_week_col', 'acc_cases_100k','acc_deaths_1mi', 'epi_week_date']
    print(print_header("Cases, by epidemiological week"), cases_by_epi_week.info())

def import_merged():
    file_path = Path('./OUT/')
    global movement_bra_merged
    global movement_bra_by_epi_week_merged
    global cases_merged
    global cases_by_epi_week_merged
    
    #movement range
    file_name = 'CVGI_FB_movement_bra_merged.csv'
    url = file_path / file_name
    movement_bra_merged = pd.read_csv(url, encoding="utf_8",parse_dates=['date'],
                                       dtype = {'GADM_GID_2':str, 'epi_week':str, 'movement_change':np.float64,
                                                'stay_put':np.float64}, engine='c')
    movement_bra_merged.drop("Unnamed: 0", axis=1, inplace=True)
    print(print_header("Movement Range, merged"), movement_bra_merged.info())
    
    file_name = 'CVGI_FB_movement_bra_by_epi_week_merged.csv'
    url = file_path / file_name
    movement_bra_by_epi_week_merged = pd.read_csv(url, encoding="utf_8", 
                                                  dtype = {'GADM_GID_2':str, 'epi_week':str,'movement_change':np.float64,
                                                           'stay_put':np.float64, 'epi_week_col':str}, engine='c')
    movement_bra_by_epi_week_merged.drop("Unnamed: 0", axis=1, inplace=True)
    print(print_header("Movement Range, by epidemiological week"), movement_bra_by_epi_week_merged.info())
    
    #cases
    file_name = 'CVGI_FB_cases_recoded.csv'
    url = file_path / file_name
    cases_merged = pd.read_csv(url, encoding="utf_8",parse_dates=['date'],dtype = {'city_ibge_code':str, 'epi_week':str}, engine='c')
    cases_merged.drop("Unnamed: 0", axis=1, inplace=True)
    print(print_header("Cases, recoded"), cases_merged.info())
    
    file_name = 'CVGI_FB_cases_by_epi_week.csv'
    url = file_path / file_name
    cases_by_epi_week_merged = pd.read_csv(url, encoding="utf_8", 
                                           dtype = {'city_ibge_code':str, 'epi_week':str, 'acc_cases':int, 'acc_deaths':int, 'new_cases':int,'new_deaths':int, 
                                                    'pop_2019': np.float64, 'epi_week_col':str, 'acc_cases_100k': np.float64,'acc_deaths_1mi': np.float64}, engine='c')
    cases_by_epi_week_merged.dropna(axis=0, inplace=True, how="all",subset=['city_ibge_code'])
    
    cases_by_epi_week_merged.columns = ['ibge_code', 'epi_week', 'acc_cases', 'acc_deaths', 'new_cases','new_deaths', 'pop_2019',
                                        'epi_week_col', 'acc_cases_100k','acc_deaths_1mi']
    print(print_header("Cases, by epidemiological week"), cases_by_epi_week_merged.info())

def import_cleaned():
    file_path = Path('./IN')
    global movement_bra_by_epi_week_merged
    global cases_by_epi_week_merged
    global movement
    global cases
    global svi_mhdi
    file_name = 'CVGI_FB_movement_bra_by_epi_week_merged.csv'
    url = file_path / file_name
    movement_bra_by_epi_week_merged = pd.read_csv(url, encoding="utf_8", 
                                                  dtype = {'GADM_GID_2':str, 'epi_week':str,'movement_change':np.float64,
                                                           'stay_put':np.float64, 'epi_week_col':str}, engine='c')
    movement_bra_by_epi_week_merged.drop("Unnamed: 0", axis=1, inplace=True)
    movement = movement_bra_by_epi_week_merged
    print(print_header("Movement Range, by epidemiological week"), movement.info())
    
    file_name = 'CVGI_FB_cases_by_epi_week.csv'
    url = file_path / file_name
    cases_by_epi_week_merged = pd.read_csv(url, encoding="utf_8", 
                                           dtype = {'city_ibge_code':str, 'epi_week':str, 'acc_cases':int, 'acc_deaths':int, 'new_cases':int,'new_deaths':int, 
                                                    'pop_2019': np.float64, 'epi_week_col':str, 'acc_cases_100k': np.float64,'acc_deaths_1mi': np.float64}, engine='c')
    cases_by_epi_week_merged.dropna(axis=0, inplace=True, how="all",subset=['city_ibge_code'])
    
    cases_by_epi_week_merged.columns = ['ibge_code', 'epi_week', 'acc_cases', 'acc_deaths', 'new_cases','new_deaths', 'pop_2019',
                                        'epi_week_col', 'acc_cases_100k','acc_deaths_1mi']

    cases = clean_cases_by_epi_week(cases_by_epi_week_merged)
    print(print_header("Cases, by epidemiological week"), cases.info())
    
    file_name = 'IPEA_BRA_IVS_MHDI_2010 municipalities.xlsx'
    url = file_path / file_name
    svi_mhdi = pd.read_excel(url, sheet_name='SVI_MHDI', dtype = {'city_name':str, 'UF':str,'ibge_cod':str,'GADM_GID_2':str,
                                                           'pop_2020':np.int64,'MHDI_tot':np.float64,
                                                           'MHDI_long':np.float64,'MHDI_edu':np.float64,'MHDI_inc':np.float64,
                                                           'SVI_tot':np.float64,'SVI_inf':np.float64,'SVI_hum':np.float64,
                                                           'SVI_wrk':np.float64,'SVI_quantiles':np.float64}) #, engine='c'
    svi_mhdi['reg'] = svi_mhdi.ibge_cod.str[0]
    #svi_mhdi.drop("Unnamed: 0", axis=1, inplace=True)
    print(print_header("IPEA SVI and MHDI data"), svi_mhdi.info())
    

<h2>Drops</h2>

In [10]:
def drops():
    drop_movement()
    drop_brasilio()
    #create_movement_cases()
    #create_cases_case_studies()
    #drop_responses_souza()
    #drop_responses_barberia()
    #drop_gadm_code()
    #time_series_clip()

def drop_movement():
    global movement_bra
    #dropping all rows from the Movement Range dataset but Brazilian ones
    movement.drop(['baseline_name','baseline_type','polygon_source'],axis='columns', inplace=True)
    movement.columns = ['date', 'country', 'GADM_GID_2', 'mun_name','movement_change','stay_put']
    movement_bra = movement[movement.country == "BRA"]
    movement_bra['date'] = pd.to_datetime(movement_bra['date'],format='%Y-%m-%d')
    print(print_header("Movement Range, dropped"), movement_bra.info())
    
    
def drop_brasilio():
    #dropping all rows from the Brasil.io dataset that are not from municipalities
    global cases
    cases_temp = cases.loc[pd.notna(cases["city"]),:]
    cases_temp['date'] = pd.to_datetime(cases_temp['date'],format='%Y-%m-%d')
    cases_temp['city_ibge_code'] = cases_temp['city_ibge_code'].astype('Int64').astype(str)    
    #Eliminating entries for states ('city' is null)
    cases_temp.dropna(axis=0, inplace=True, how="all",subset=['last_available_confirmed','last_available_confirmed_per_100k_inhabitants', 'last_available_date','last_available_death_rate', 'last_available_deaths', 'order_for_place','place_type', 'state', 'new_confirmed', 'new_deaths'])
    #Eliminate unused columns ['order_for_place','place_type','last_available_date']
    cases_temp.drop(['order_for_place','place_type','last_available_date','estimated_population','state','last_available_confirmed_per_100k_inhabitants','last_available_death_rate'], axis=1, inplace=True)
    cases = cases_temp
    cases.columns = ['mun_name', 'ibge_code', 'date', 'epidemiological_week','pop_2019', 'is_last', 'is_repeated','acc_cases', 'acc_deaths', 'new_cases','new_deaths']
    print(print_header("Cases, dropped"), cases.info())
    
def create_movement_cases():
    global movement_rj
    global movement_sp
    global movement_pa
    global movement_case_studies
    movement_rj = movement_bra[movement_bra.GADM_GID_2 == "BRA.19.67_1"]
    movement_sp = movement_bra[movement_bra.GADM_GID_2 == "BRA.25.564_1"]
    movement_pa = movement_bra[movement_bra.GADM_GID_2 == "BRA.21.305_1"]
    movement_case_studies = pd.concat([movement_rj,movement_sp,movement_pa])
    movement_bra.drop(['country'],axis=1,inplace=True)
    print(print_header("Movement Range, case studies"), movement_case_studies.info())
    
def create_cases_case_studies():
    global cases_case_studies
    global cases_rj
    global cases_sp
    global cases_pa
    cases_rj = cases[cases.ibge_code == '3304557']
    cases_sp = cases[cases.ibge_code == '3550308']
    cases_pa = cases[cases.ibge_code == '4314902']
    cases_case_studies = pd.concat([cases_rj,cases_sp,cases_pa])
    print(print_header("COVID cases, case studies"), cases_case_studies.info())
    
def drop_responses_souza():
    global responses_souza
    responses_souza.dropna(axis=0, inplace=True, how="all",subset=['Q1. Cordon Sanitaire (monitoring of entrance and exit of people in the municipality)','Q1. Start date','Q2. Restrictive measures to avoid circulation/ agglomeration of people','Q2. Start date','Q3. Measures of social isolation, allowing ONLY essential services','Q3. Start date', 'Q4. Compulsory use of face covers', 'Q4. Start date','Q5. Were any measures implemented to reduce the offer of public transportation?','Q5. What was the percentage of reduction?', 'Q5. Start date','Q6. Were measures of restriction and social isolation eased?','Q6. Start date'])
    print(print_header("Responses (Souza et al.), dropped"), responses_souza.info())    
    
def drop_responses_barberia():
    global responses_barberia
    responses_barberia["city_name"] = responses_barberia.city + "/" + responses_barberia.state
    print(print_header("Responses (Barberia et al.), dropped"), responses_barberia.info())    
    
def drop_gadm_code():
    global gadm_code
    gadm_code.dropna(axis=1,inplace=True, how="all")
    print(print_header("GADM Codes, dropped"), gadm_code.info())    


def time_series_clip():
    global cases_case_studies
    global movement_case_studies
    begin_movement = movement_case_studies.ds.min()
    end_movement = movement_case_studies.ds.max()
    print("Begin move: ", begin_movement, "\nEnd move: ", end_movement)

    begin_cases = cases_case_studies.date.min()
    end_cases = cases_case_studies.date.max()
    print("Begin cases: ", begin_cases, "\nEnd cases: ", end_cases)

    if begin_movement > begin_cases:
        begin_date = begin_movement
        begin_early = "cases_case_studies" #I need to clip the dataset that has earlier dates

    else:
        begin_date = begin_cases
        begin_early = "movement_case_studies"
    print("Begins earlier:",begin_early, "(should be clipped)")
    print(begin_date)

    if end_movement < end_cases:
        end_date = end_movement
        end_later = "cases_case_studies"
    else:
        end_date = end_cases
        end_later = "movement_case_studies"
    print("Ends later:",end_later, "(should be clipped)")
    print(begin_date)

    if begin_early == "cases_case_studies":
        cases_case_studies = cases_case_studies[cases_case_studies.date>=begin_date]
        print("cases are clipped at", cases_case_studies.date.min())
    elif begin_early == "movement_case_studies":
        movement_case_studies = movement_case_studies[movement_case_studies.ds>=begin_date]
        print("movements are clipped at", movement_case_studies.ds.min())

    if end_later == "cases_case_studies":
        cases_case_studies = cases_case_studies[cases_case_studies.date<=end_date]
        print("cases are clipped at", cases_case_studies.date.max())
    elif end_later == "movement_case_studies":
        movement_case_studies = movement_case_studies[movement_case_studies.ds<=end_date]
        print("movements are clipped at", movement_case_studies.ds.max())

<h2>cleanup</h2>

In [2]:
def clean_movement_bra(rows = None, to_csv = False):
    #function takes two parameters:
    #rows = a filter for the rows of the dataset, can receive an interval or the "randSample" variable
    #to_csv = selects whether "move_bra_week" is exported to a CSV at the end
    global movement_bra
    global move_bra_week
    if rows == None:
        rows =  range(len(movement_bra.index))
    movement_bra = new_columns(movement_bra,['epi_week'])
    for i in range(len(movement_bra.iloc[rows])):
        week_txt = str(movement_bra.ds.iloc[i].isocalendar()[1])
        year_txt = str(movement_bra.ds.iloc[i].isocalendar()[0])
        if week_txt < 2:
            week_txt = "0" + week_txt 
        movement_bra.epi_week.iloc[i] = year_txt + '.' + week_txt
    move_bra_week = movement_bra.groupby(['GADM_GID_2','epi_week']).mean()
    move_bra_week['epi_week_col'] = move_bra_week.index
    for i in range(len(move_bra_week)):
        move_bra_week.epi_week_col[i] = move_bra_week.epi_week_col[i][1]
    if to_csv == True:
        file_path = Path('./OUT/')
        file_name = 'CVGI_FB_movement_bra_by_epi_week.csv'
        url = file_path / file_name
        move_bra_week.to_csv(url, encoding='utf-8-sig',)        
    return move_bra_week

def cases_cleanup(rows = None, do_merge = False, to_csv = False):
    global cases
    global cases_week
    if rows == None:
        rows =  range(len(cases.index))
    cases = new_columns(cases,['epi_week'])
    #print(rows)    
    for i in range(len(cases.iloc[rows])):
        week_txt = right(str(cases.epidemiological_week.iloc[i]),2) #Brasil.io always codes "epidemiological_week" with 6 digits
        year_txt = left(str(cases.epidemiological_week.iloc[i]),4)
        cases.epi_week.iloc[i] = year_txt + "." + week_txt
    if do_merge == False:
        return cases
    elif do_merge == True:
        #we'll have two new datasets: one for the variables that are summed while groupping
        #another for the variables that are averaged out while grouping
        #sum temp dataset, removes columns we'll not use
        #we are left with the columns: 'new_confirmed', 'new_deaths', 'acc_cases', 'acc_deaths'
        cases_temp_sum = cases[['ibge_code','epi_week','new_confirmed','new_deaths','acc_cases','acc_deaths']]
        cases_week_sum = cases_temp_sum.groupby(['ibge_code','epi_week']).sum()
        #mean temp dataset, we just keep one column, makes sense to slice 
        cases_temp_mean = cases[["ibge_code","epi_week","pop_2019"]]
        cases_week_mean = cases_temp_mean.groupby(['ibge_code','epi_week']).mean()

        #merging the datasets
        cases_week = cases_week_sum.merge(cases_week_mean, how='left',left_index=True,right_index=True)
        cases_week['epi_week_col'] = cases_week.index
        for i in range(len(cases_week)):
            cases_week.epi_week_col[i] = cases_week.epi_week_col[i][1]
        
        #calculating rates of cases per 100k people and deaths per million
        cases_week['acc_cases_100k'] = round(cases_week.acc_cases / (cases_week.pop_2019 / 100000),2)
        cases_week['acc_deaths_1mi'] = round(cases_week.acc_deaths / (cases_week.pop_2019 / 1000000),2)
        
    if to_csv == True:
        file_path = Path('./OUT/')
        file_name = 'CVGI_FB_cases_by_epi_week.csv'
        url = file_path / file_name
        cases_week.to_csv(url, encoding='utf-8-sig')        
    return cases_week

def clean_cases_by_epi_week(df):
    #global cases_by_epi_week
    df['epi_week_date'] = ".0"
    df['epi_week_date'] = df.epi_week.astype("str").str.pad(7, side='right', fillchar='0') + df.epi_week_date
    df['epi_week_date'] = pd.to_datetime(df['epi_week_date'],format='%Y.%W.%w')
    return df

In [ ]:
ibge_uf_dict = {11:"RO",12:"AC",13:"AM",14:"RR",15:"PA",16:"AP",17:"TO",
                21:"MA",22:"PI",23:"CE",24:"RN",25:"PB",26:"PE",27:"AL",28:"SE",29:"BA",
                31:"MG",32:"ES",33:"RJ",35:"SP",
                41:"PR",42:"SC",43:"RS",
                50:"MS",51:"MT",52:"GO",53:"DF"}

---

<h2> Kaplan-Meier plot and upscaling functions</h2>

Upscaling function city level:

In [3]:
def events_cities(cases):
    start_time = time.time()
    total_deaths = 0
    for id_wk, val_wk in enumerate(cases.epi_week):
        #check if there are any cases, otherwise skip
        var_deaths = cases[(cases.epi_week == val_wk)].new_deaths.tolist()[0]
        #check if the death number of a city is positive. we are ignoring the negative numbers.
        if var_deaths > 0: total_deaths = total_deaths + var_deaths
    events = pd.DataFrame({'event': pd.Series([], dtype='int'),'week':pd.Series([], dtype='int')}, index=range(total_deaths))
    c = 0
    for id_wk, val_wk in enumerate(cases.epi_week):
        #check if there are any cases, otherwise skip
        var_deaths = cases[(cases.epi_week == val_wk)].new_deaths.tolist()[0]
        #ignore negative death numbers (they are low, and apply to very few cases, see if this needs to be improved)
        if var_deaths > 0:
            for i in range(var_deaths):
                events['event'].iloc[c] = 1 
                events['week'].iloc[c] = id_wk
                c = c + 1
    t1 = "--- %s seconds ---" % (time.time() - start_time)
    return events

Upscaling for multiple cities

In [ ]:
def events_mc(cities_list, dates=None):
    #cases_uf = cases[(cases['ibge_code'].str.startswith(uf)) & (cases['epi_week_date'] < ('2021-02-25'))]
    unique_cities = cities_list
    dict_events_mc = {}
    
    if dates == None:
        for ibge_code in unique_cities:
            #print(ibge_code)
            cases_test = cases[cases.ibge_code == ibge_code][['epi_week','new_deaths']]
            dict_events_mc[ibge_code] = events_cities(cases_test)
    #function returns a dict, it can be transformed in a dataframe using concat
    else:
        for ibge_code in unique_cities:
            #print(ibge_code)
            cases_test = cases[(cases.ibge_code == ibge_code)&(cases.epi_week_date < dates)][['epi_week','new_deaths']]
            dict_events_mc[ibge_code] = events_cities(cases_test)
    df_events = pd.concat(dict_events_mc)
    return df_events

Upscaling function state level:

In [4]:
def events_uf(uf = '11'):
    #uf is a string! don't forget
    cases_uf = cases[(cases['ibge_code'].str.startswith(uf)) & (cases['epi_week_date'] < ('2021-02-25'))]
    unique_cities = cases_uf.ibge_code.unique()
    #print(unique_cities)
    dict_events_uf = {}
    
    for ibge_code in unique_cities:
        #print(ibge_code)
        cases_test = cases[cases.ibge_code == ibge_code][['epi_week','new_deaths']]
        dict_events_uf[ibge_code] = events_cities(cases_test)
    #function returns a dict, it can be transformed in a dataframe using concat
    df_events = pd.concat(dict_events_uf)
    return df_events

Kaplan-Meier plot

In [4]:
def plot_kmf(dataframe,cities):
    kmf = KaplanMeierFitter() 
    df = dataframe
    #list currently takes a max of 5 cities because of the ax elements in the plot at the end
    cities_list = cities

    T = df['week']     ## time to event
    E = df['event']      ## event occurred or censored


    #creating a the cohorts. this takes a list of index (named) keys
    groups = df.loc[cities_list]
    #create an empty series object to receive the dataframe for each city below
    ix = pd.Series([])
    for i in range(len(cities_list)):
        #create a temporary dataframe, holding the events for each city
        ix[i] = groups.loc[cities_list[i]].index
        cities_label = cities_list[i]
        #cities_label = ibge_codes[ibge_codes.ibge_code == cities_list[i]].mun_name
        #passing on the dataframe's time and event columns to the Kaplan-Meier function, and the index keys as labels (we can search the IBGE database for its name later)
        kmf.fit(T[ix[i]], E[ix[i]], label=cities_label) 
        figsize(14,7)
        if i == 0:
            ax = kmf.plot()
        elif i > 0:
            ax1 = kmf.plot(ax=ax)

    plt.show()
    return kmf